In [93]:
import torch

In [94]:
torch.cuda.is_available()

False

In [95]:
torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

In [96]:
torch.cuda.mem_get_info()

AssertionError: Torch not compiled with CUDA enabled

In [97]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

if device.type=='cuda':
    print(torch.cuda.get_device_name(0))

cpu


In [98]:
CLASSES=10
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [99]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
#import matplotlib.pyplot as plt
import numpy as np

In [100]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        #Layer 1
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=96,kernel_size=11,stride=4)

        #Layer 2, make sure to apply RELU in forward
        self.conv2 = nn.Conv2d(in_channels = 96, out_channels = 96,kernel_size = 1)
        #Layer 3, Make sure to apply RELU, maxpool2d, and dropout of 0.5
        self.conv3 = nn.Conv2d(in_channels = 96, out_channels = 96,kernel_size = 1)
        self.pool1 = nn.MaxPool2d(kernel_size = 3, stride = 2)
        #Layer4: Apply RELU
        self.conv4 = nn.Conv2d(in_channels = 96, out_channels = 256,kernel_size = 11,stride = 4, padding = 2)

        #Layer 5a APply ReLU
        self.conv5a = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 1)

        #Layer5, Apply ReLU, Maxpool, and dropout
        self.conv5 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size=1)
        self.pool2 = nn.MaxPool2d(kernel_size = 3, stride =2)
        

        #Layer 6: Apply ReLU
        self.conv6 = nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size=3,stride = 1, padding = 1)

        #Layer 7: Apply ReLU
        self.conv7 = nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size=1)

        #Layer 8 Apply ReLU and dropout of 0.5
        self.conv8 = nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size=1)

        #Layer 9 Apply ReLU
        self.conv9 = nn.Conv2d(in_channels = 384, out_channels = 10, kernel_size=3,stride = 1, padding = 1)

        #Layer 10 Apply ReLU
        self.conv10 = nn.Conv2d(in_channels = 10, out_channels = 10, kernel_size=1)

        #Layer 11 Apply ReLU, and apply nn.AdaptiveAvgPool2d
        self.conv11 = nn.Conv2d(in_channels = 10, out_channels = 10,kernel_size = 1)
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))


    def forward(self, x):
        #Layer 1
        x = F.relu(self.conv1(x))
        #Layer 2
        x = F.relu(self.conv2(x))
        #Layer 3
        x = self.pool1(F.relu(self.conv3(x)))
        x = F.dropout(x,0.5)
        #Layer 4
        x = F.relu(self.conv4(x))
        
        #Layer 5a
        x = F.relu(self.conv5a(x))
        #Layer 5
        x = F.dropout(x,0.5)
        x = self.pool2(F.relu(self.conv5(x)))
        #Layer 6
        x = F.relu(self.conv6(x))
        #Layer 7
        x = F.relu(self.conv7(x))
        #Layer 8
        x = F.relu(self.conv8(x))
        x = F.dropout(x,0.5)
        #Layer 9
        x = F.relu(self.conv9(x))
        #Layer 10
        x = F.relu(self.conv10(x))
        #Layer 11
        x = F.relu(self.conv11(x))
        x = self.avg_pool(x)
        x = torch.flatten(x,1)
        return x

In [101]:
transform_conf=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [102]:
BATCH_SIZE=16
train_dataset=datasets.FashionMNIST(root='./data/',train=True,download=True,transform=transform_conf,)
test_dataset=datasets.FashionMNIST(root = './data/',train=False,download=True,transform=transform_conf)

In [103]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=True)

In [104]:
model=Net().to(device)
optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [105]:
def train(model,device,train_loader,optimizer,epochs):
    print("inside train")
    model.train()
    for batch_ids, (img, classes) in enumerate(train_loader):
        classes=classes.type(torch.LongTensor)
        img,classes=img.to(device),classes.to(device)
        torch.autograd.set_detect_anomaly(True)     
        optimizer.zero_grad()
        output=model(img)
        loss = loss_fn(output,classes)                
        
        loss.backward()
        optimizer.step()
    if(batch_ids +1) % 2 == 0:
        print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epochs, batch_ids* len(img), len(train_loader.dataset),
            100.*batch_ids / len(train_loader),loss.item()))

In [106]:
def test(model, device, test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for img,classes in test_loader:
            img,classes=img.to(device), classes.to(device)
            y_hat=model(img)
            test_loss+=F.nll_loss(y_hat,classes,reduction='sum').item()
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==classes).sum().item()
        test_loss/=len(test_dataset)
        print("\n Test set: Avarage loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
            test_loss,correct,len(test_dataset),100.*correct/len(test_dataset)))
        print('='*30)

In [ ]:
if __name__=='__main__':
    seed=42
    EPOCHS=1
    
    for epoch in range(1,EPOCHS+1):
        train(model,device,train_loader,optimizer,epoch)
        test(model,device,test_loader)

inside train
